In [ ]:
!pip install wordcloud nltk  pyldavis > /dev/null

In [ ]:
import pandas as pd
papers = pd.read_csv("/content/drive/MyDrive/Pós Live/MIT de Inteligencia Artificial e Machine Learning/202204 - Processamento de linguagem natural com Python/notebooks/data/papers.csv")

In [ ]:
papers.head()

id  year                                              title event_type  \
0     1  1987  Self-Organization of Associative Database and ...        NaN   
1    10  1987  A Mean Field Theory of Layer IV of Visual Cort...        NaN   
2   100  1988  Storing Covariance by the Associative Long-Ter...        NaN   
3  1000  1994  Bayesian Query Construction for Neural Network...        NaN   
4  1001  1994  Neural Network Ensembles, Cross Validation, an...        NaN   

                                            pdf_name          abstract  \
0  1-self-organization-of-associative-database-an...  Abstract Missing   
1  10-a-mean-field-theory-of-layer-iv-of-visual-c...  Abstract Missing   
2  100-storing-covariance-by-the-associative-long...  Abstract Missing   
3  1000-bayesian-query-construction-for-neural-ne...  Abstract Missing   
4  1001-neural-network-ensembles-cross-validation...  Abstract Missing   

                                          paper_text  
0  767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...  
1  683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...  
2  394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...  
3  Bayesian Query Construction for Neural\nNetwor...  
4  Neural Network Ensembles, Cross\nValidation, a...

In [ ]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")

def tokenize(text):
    text = text.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(text)
    return ' '.join(new_words)

papers['paper_text_processed'] = papers.paper_text.apply(tokenize)
papers['paper_text_processed'].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    767 self organization of associative database ...
1    683 a mean field theory of layer iv of visual ...
2    394 storing covariance by the associative long...
3    bayesian query construction for neural network...
4    neural network ensembles cross validation and ...
Name: paper_text_processed, dtype: object

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords.words('english'),
                        min_df=10,
                        max_features=5000)
tfidf.fit(papers['paper_text_processed'])
papers.loc[:, 'tfidf'] = papers['paper_text_processed'].apply(lambda t: tfidf.transform([t]).toarray()[0])



In [ ]:
papers.head()

id  year                                              title event_type  \
0     1  1987  Self-Organization of Associative Database and ...        NaN   
1    10  1987  A Mean Field Theory of Layer IV of Visual Cort...        NaN   
2   100  1988  Storing Covariance by the Associative Long-Ter...        NaN   
3  1000  1994  Bayesian Query Construction for Neural Network...        NaN   
4  1001  1994  Neural Network Ensembles, Cross Validation, an...        NaN   

                                            pdf_name          abstract  \
0  1-self-organization-of-associative-database-an...  Abstract Missing   
1  10-a-mean-field-theory-of-layer-iv-of-visual-c...  Abstract Missing   
2  100-storing-covariance-by-the-associative-long...  Abstract Missing   
3  1000-bayesian-query-construction-for-neural-ne...  Abstract Missing   
4  1001-neural-network-ensembles-cross-validation...  Abstract Missing   

                                          paper_text  \
0  767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...   
1  683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...   
2  394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...   
3  Bayesian Query Construction for Neural\nNetwor...   
4  Neural Network Ensembles, Cross\nValidation, a...   

                                paper_text_processed  \
0  767 self organization of associative database ...   
1  683 a mean field theory of layer iv of visual ...   
2  394 storing covariance by the associative long...   
3  bayesian query construction for neural network...   
4  neural network ensembles cross validation and ...   

                                               tfidf  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0179797964259828, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.02265883504480681, 0.04534999223346446, 0.0...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008...

In [ ]:
corpus = np.array(papers.tfidf.tolist())


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
import numpy as np

lda = LDA(n_components=5, random_state=42).fit(corpus)

In [ ]:
def get_model_topics(model, vectorizer, n_top_words=10):
    word_dict = {}
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        word_dict[topic_idx] = top_features

    return pd.DataFrame(word_dict)

get_model_topics(lda, tfidf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


0          1         2               3            4
0         policy  algorithm     model            stdp        skill
1         reward       data   network           sleep       skills
2         regret   learning  learning           dayan   temporally
3         action     matrix     image             vae        intra
4          agent        set    neural           spike         room
5        actions   function  training          spikes         head
6  reinforcement        log  networks       attractor   navigation
7          state     kernel     input   connectionist          cij
8       policies         xi      time      temporally  termination
9           game      model   neurons  spatiotemporal         year

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()



/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
corpus_matrix = np.matrix(corpus)

pyLDAvis.sklearn.prepare(lda, corpus_matrix, tfidf)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.077927 -0.069966       1        1  53.905013
2     -0.082736  0.029157       2        1  40.120361
0      0.204458 -0.009480       3        1   3.223098
4     -0.022153  0.026212       4        1   1.379752
3     -0.021642  0.024076       5        1   1.371776, topic_info=            Term        Freq       Total Category  logprob  loglift
3397      policy  123.000000  123.000000  Default  30.0000  30.0000
4297       state  137.000000  137.000000  Default  29.0000  29.0000
3753      regret   81.000000   81.000000  Default  28.0000  28.0000
293       action   82.000000   82.000000  Default  27.0000  27.0000
3873      reward   74.000000   74.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
4129     silicon    0.269642   10.264823   Topic5  -8.5159   0.6497
4532         thm    0.269634    7.212450   Topic5  -8.5160   1.0026
4050     sensory    0.269632   25.573654   Topic5  -8.5160  -0.2632
2933   movements    0.269632   16.886490   Topic5  -8.5160   0.1519
1693  excitation    0.269631    9.166985   Topic5  -8.5160   0.7628

[341 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
16        1  0.666696      10
16        2  0.330593      10
56        1  0.543874     175
56        2  0.362583     175
293       1  0.012187  action
...     ...       ...     ...
4954      2  0.375342     yan
4958      1  0.518721    year
4958      2  0.389041    year
4962      1  0.876569      yi
4962      2  0.116016      yi

[511 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 5, 4])